**Title**

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# !pip install transformers
!pip install datasets
!pip install evaluate

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM, BartForConditionalGeneration
from datasets import load_dataset
import torch
from torch.utils.data import DataLoader
import numpy as np
# AutoModelForConditionalGeneration

BART
* Base model has no language head (output is embeddings)
* CausalLM is decoder-only but not all weights are pretrained
* ConditionalGeneration is the same as the base model but includes the language head

In [ ]:
model_name = 'facebook/bart-base'
model_name = 'facebook/bart-large-cnn'
# model_name = 'gpt2-medium'
# model_name = 'facebook/bart-large'
# model = AutoModel.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# print(model)
dir(model)
model.config
model.config.max_position_embeddings # maximum input sequence length

In [ ]:
input_text = r"Medical Specialty:\
General Medicine\
Sample Name: Abdominal Pain - Consult\
Description: The patient presented to the emergency room last evening with approximately 7- to 8-day history of abdominal pain which has been persistent.\
(Medical Transcription Sample Report)\
CHIEF COMPLAINT: Abdominal pain.\
HISTORY OF PRESENT ILLNESS: The patient is a 71-year-old female patient of Dr. X. The patient presented to the emergency room last evening with approximately 7- to 8-day history of abdominal pain which has been persistent. She was seen 3 to 4 days ago at ABC ER and underwent evaluation and discharged and had a CT scan at that time and she was told it was 'normal.' She was given oral antibiotics of Cipro and Flagyl. She has had no nausea and vomiting, but has had persistent associated anorexia. She is passing flatus, but had some obstipation symptoms with the last bowel movement two days ago. She denies any bright red blood per rectum and no history of recent melena. Her last colonoscopy was approximately 5 years ago with Dr. Y. She has had no definite fevers or chills and no history of jaundice. The patient denies any significant recent weight loss.\
PAST MEDICAL HISTORY: Significant for history of atrial fibrillation, under good control and now in normal sinus rhythm and on metoprolol and also on Premarin hormone replacement.\
PAST SURGICAL HISTORY: Significant for cholecystectomy, appendectomy, and hysterectomy. She has a long history of known grade 4 bladder prolapse and she has been seen in the past by Dr. Chip Winkel, I believe that he has not been re-consulted.\
ALLERGIES: SHE IS ALLERGIC OR SENSITIVE TO MACRODANTIN.\
SOCIAL HISTORY: She does not drink or smoke.\
REVIEW OF SYSTEMS: Otherwise negative for any recent febrile illnesses, chest pains or shortness of breath.\
PHYSICAL EXAMINATION:\
GENERAL: The patient is an elderly thin white female, very pleasant, in no acute distress.\
VITAL SIGNS: Her temperature is 98.8 and vital signs are all stable, within normal limits.\
HEENT: Head is grossly atraumatic and normocephalic. Sclerae are anicteric. The conjunctivae are non-injected.\
NECK: Supple.\
CHEST: Clear.\
HEART: Regular rate and rhythm.\
ABDOMEN: Generally nondistended and soft. She is focally tender in the left lower quadrant to deep palpation with a palpable fullness or mass and focally tender, but no rebound tenderness. There is no CVA or flank tenderness, although some very minimal left flank tenderness.\
PELVIC: Currently deferred, but has history of grade 4 urinary bladder prolapse.\
EXTREMITIES: Grossly and neurovascularly intact.\
LABORATORY VALUES: White blood cell count is 5.3, hemoglobin 12.8, and platelet count normal. Alkaline phosphatase elevated at 184. Liver function tests otherwise normal. Electrolytes normal. Glucose 134, BUN 4, and creatinine 0.7.\
DIAGNOSTIC STUDIES: EKG shows normal sinus rhythm.\
IMPRESSION AND PLAN: A 71-year-old female with greater than one-week history of abdominal pain now more localized to the left lower quadrant. Currently is a nonacute abdomen. The working diagnosis would be sigmoid diverticulitis. She does have a history in the distant past of sigmoid diverticulitis. I would recommend a repeat stat CT scan of the abdomen and pelvis and keep the patient nothing by mouth. The patient was seen 5 years ago by Dr. Y in Colorectal Surgery. We will consult her also for evaluation. The patient will need repeat colonoscopy in the near future and be kept nothing by mouth now empirically. The case was discussed with the patient's primary care physician, Dr. X. Again, currently there is no indication for acute surgical intervention on today's date, although the patient will need close observation and further diagnostic workup."

In [ ]:
input_text = r"This time, Icarus had followed his dad’s instructions.\
He’d taxied along the runway and was about to take off when his earbuds had fallen out and he’d lost contact with air traffic control. His legs were almost exhausted.\
Then, a bit of luck. A jumbo jet had assisted his take-off, boosting him into the air. The pilot had gesticulated that Icarus should get off the nose cone but that was easier said than done, although finally, he’d freed himself and plummeted to earth.\
​Thankfully, his fall was broken by the softened wax and broken feathers.\
Back to the drawing board."

In [ ]:
# input_text = "Once upon a time"
# input_text = 'Create a story from this prompt: once upon a time'
input_ids = tokenizer.encode(input_text, return_tensors="pt")
output = model.generate(input_ids, max_length=1000)
print(tokenizer.decode(output[0], skip_special_tokens=True))
# print(tokenizer.decode(output[0], skip_special_tokens=False))

This time, Icarus had followed his dad’s instructions. His earbuds had fallen out and he’d lost contact with air traffic control. A jumbo jet had assisted his take-off, boosting him into the air. The pilot had gesticulated that Icarus should get off the nose cone but that was easier said than done.


In [ ]:
# https://huggingface.co/datasets/har1/MTS_Dialogue-Clinical_Note?row=8
# All the data is in the 'train' split, but the documentation says 1201 training samples, 100 validation samples
ds = load_dataset("har1/MTS_Dialogue-Clinical_Note", streaming=False)

README.md:   0%|          | 0.00/1.67k [00:00<?, ?B/s]

MTS-Dialog-TrainingSet (SDHP).csv:   0%|          | 0.00/1.02M [00:00<?, ?B/s]

MTS-Dialog-Validation Set (SDHP).csv:   0%|          | 0.00/74.8k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1301 [00:00<?, ? examples/s]

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['ID', 'section_header', 'section_text', 'dialogue'],
        num_rows: 1301
    })
})

In [ ]:
ds['train'][0]['section_text']
# ds['train'][0]['dialogue']
# ds['train'][0]

'Symptoms: no fever, no chills, no cough, no congestion, no nausea, no vomiting, no chest pain, no chest pressure.\nDiagnosis: hypertension, osteoarthritis, osteoporosis, hypothyroidism, allergic rhinitis, kidney stones\nHistory of Patient: 76-year-old white female, presents to the clinic today originally for hypertension and a med check, followed by Dr. Kumar, issues stable\nPlan of Action: N/A'

In [ ]:
tokens = tokenizer.tokenize(ds['train'][0]['section_text'])
ids = tokenizer.convert_tokens_to_ids(tokens)
decoded_string = tokenizer.decode(ids)
print(decoded_string)

Symptoms: no fever, no chills, no cough, no congestion, no nausea, no vomiting, no chest pain, no chest pressure.
Diagnosis: hypertension, osteoarthritis, osteoporosis, hypothyroidism, allergic rhinitis, kidney stones
History of Patient: 76-year-old white female, presents to the clinic today originally for hypertension and a med check, followed by Dr. Kumar, issues stable
Plan of Action: N/A


In [ ]:
dir(tokenizer)
tokenizer

BartTokenizerFast(name_or_path='facebook/bart-large-cnn', vocab_size=50265, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	50264: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True, special=True),
}

In [ ]:
tokenizer_out = tokenizer(ds['train'][0]['section_text'], return_tensors='pt')
tokenizer_out

{'input_ids': tensor([[    0, 46994, 47629,    35,   117, 11696,     6,   117,  1855,  5622,
             6,   117, 21768,     6,   117, 15744,     6,   117, 27214,     6,
           117, 23600,     6,   117,  7050,  2400,     6,   117,  7050,  1164,
             4, 50118, 29038, 11244, 13310,    35, 30960,     6, 30311,   139,
         16880, 37830,     6, 30311,  1517,   368, 13310,     6, 15671, 37994,
         36866,  1809,     6, 28349, 16448,   179, 10100,     6, 12855, 15109,
         50118, 38261,     9, 27690,    35,  5553,    12,   180,    12,   279,
          1104,  2182,     6,  6822,     7,     5,  8474,   452,  3249,    13,
         30960,     8,    10,  5679,  1649,     6,  1432,    30,   925,     4,
          6706,     6,   743,  4375, 50118, 35351,     9,  5828,    35,   234,
            73,   250,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [ ]:
type(tokenizer_out['input_ids'])

torch.Tensor

In [ ]:
# Class to handle tokenization and dataloading

class TextPreprocessor:
    def __init__(self, tokenizer, model, dataset, input_col, output_col=None, padding="longest", truncation=True, batch_size=8):
        """
        Initialize the class with a tokenizer, model, dataset, and various tokenization options.
        - dataset - currently assumes this is a dataset dictionary with keys 'train', 'val', 'test'
        - input_col: The column name for the input sequences.
        - output_col: The column name for the output sequences (if applicable). If None, assume single sequence input.
        - padding: Padding strategy (e.g., "max_length", "longest", or a specific integer length).
        - truncation: Truncation strategy (boolean or "longest_first").
        """
        self.tokenizer = tokenizer
        self.model = model
        self.model_max_length = model.config.max_position_embeddings
        self.dataset = dataset
        self.input_col = input_col
        self.output_col = output_col
        self.padding = padding
        self.truncation = truncation
        self.batch_size = batch_size
        self.tokenized_dataset = {}
        self.truncation_tracker = {}
        self.dataloader = {}

    def tokenize_function(self, examples):
        # Tokenize a pair of sequences (input-output pair, e.g. for summarization)
        if self.output_col:
            tokenized = self.tokenizer(
                examples[self.input_col],
                text_target=examples[self.output_col],   # target sequence
                padding=self.padding,
                truncation=self.truncation,
                max_length=self.model_max_length,
                return_tensors='np',
                return_length=True,
                # return_overflowing_tokens=True
            )

        # Tokenize a single sequence
        else:
            tokenized = self.tokenizer(
                examples[self.input_col],
                padding=self.padding,
                truncation=self.truncation,
                max_length=self.model_max_length,
                return_tensors='np',
                return_length=True,
                # return_overflowing_tokens=True
            )

        # tokenized = dictionary
        # keys = input_ids, attention_mask, labels
        # size = batch size, sequence length (input_ids and attention_mask have the same length)

        # Check for truncation
        if self.truncation not in [False, 'do_not_truncate']:
            # Get non-truncated lengths
            input_lens = self.tokenizer(examples[self.input_col], truncation=False, return_tensors='np', return_length=True)['length']
            # print(type(input_lens))
            # print(input_lens.shape)
            # print(input_lens[:10])

            # Calculate how many tokens were truncated
            trunc_lens = np.clip(input_lens - tokenized['length'], a_min=0, a_max=None)
            trunc_lens_indexing = np.nonzero(trunc_lens > 0)[0]
            # print(type(trunc_lens))
            # print(trunc_lens.shape)
            # print(trunc_lens_indexing[:10])

            # Get IDs for non-zero truncations
            # print(type(examples['ID']))
            # trunc_ids = examples['ID'][trunc_lens_indexing]
            trunc_ids = [examples['ID'][i] for i in trunc_lens_indexing]
            trunc_lens = trunc_lens[trunc_lens > 0].tolist()

            self.truncation_tracker.update(dict(zip(trunc_ids, trunc_lens)))

        return tokenized

    def tokenize_dataset(self):
        """Applies tokenization to the dataset using the tokenize_function."""
        for key in self.dataset:
            print(f'Tokenizing {key} split...')
            self.tokenized_dataset[key] = self.dataset[key].map(self.tokenize_function, batched=True)
            self.tokenized_dataset[key].set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

    def create_dataloader(self, shuffle_train=True):
        """Creates a DataLoader from the tokenized dataset."""
        if self.tokenized_dataset == {}:
            raise ValueError("Tokenized dataset is not available. Run tokenize_dataset() first.")

        # Convert to PyTorch DataLoader for batching
        for key in self.tokenized_dataset:
            shuffle = shuffle_train if key == 'train' else False
            print(f'Creating {key} dataloader...')
            self.dataloader[key] = DataLoader(self.tokenized_dataset[key], batch_size=self.batch_size, shuffle=shuffle)

    def get_dataloader(self):
        """Returns the DataLoader, creating it if necessary."""
        if self.dataloader == {}:
            self.create_dataloader()
        return self.dataloader

In [ ]:
preprocessor = TextPreprocessor(tokenizer, model, ds, 'dialogue', output_col='section_text', padding='max_length')

preprocessor.tokenize_dataset()

Tokenizing train split...


Map:   0%|          | 0/1301 [00:00<?, ? examples/s]

<class 'numpy.ndarray'>
(1000,)
[174 382 152 119  52  97 242  16 298 106]
<class 'numpy.ndarray'>
(1000,)
[297 399]
<class 'list'>
<class 'numpy.ndarray'>
(301,)
[ 16  81 929 137  85  52  30 424  50 425]
<class 'numpy.ndarray'>
(301,)
[ 96 115]
<class 'list'>


In [ ]:
print(type(preprocessor.tokenized_dataset['train']['input_ids']))
print(preprocessor.tokenized_dataset['train']['input_ids'].shape)
print(type(preprocessor.tokenized_dataset['train']['attention_mask']))
print(preprocessor.tokenized_dataset['train']['attention_mask'].shape)
print(type(preprocessor.tokenized_dataset['train']['labels']))
print(preprocessor.tokenized_dataset['train']['labels'].shape)

<class 'torch.Tensor'>
torch.Size([1301, 1024])
<class 'torch.Tensor'>
torch.Size([1301, 1024])
<class 'torch.Tensor'>
torch.Size([1301, 1024])


In [ ]:
preprocessor.create_dataloader()

Creating train dataloader...


In [ ]:
print(type(preprocessor.dataloader)) # dictionary. Contains 'train', 'val', 'test'.
print(type(preprocessor.dataloader['train'])) # dataloader


first_batch = next(iter(preprocessor.dataloader['train']))
print(type(first_batch)) # dictionary
print(first_batch.keys())
print(type(first_batch['input_ids']))
print(first_batch['input_ids'].shape)
print(type(first_batch['attention_mask']))
print(first_batch['attention_mask'].shape)
print(type(first_batch['labels']))
print(first_batch['labels'].shape)

<class 'dict'>
<class 'torch.utils.data.dataloader.DataLoader'>
<class 'dict'>
dict_keys(['input_ids', 'attention_mask', 'labels'])
<class 'torch.Tensor'>
torch.Size([8, 1024])
<class 'torch.Tensor'>
torch.Size([8, 1024])
<class 'torch.Tensor'>
torch.Size([8, 1024])


In [ ]:
# Example code from https://huggingface.co/docs/datasets/en/use_with_pytorch


import numpy as np
from datasets import Dataset
from torch.utils.data import DataLoader

# Data and labels are scalars, so when you batch with size 4, each batch consists of tensors of length 4
# data = np.random.rand(16)
# label = np.random.randint(0, 2, size=16)


# Modify data and label to be vectors of length 7 (16 total samples)
# Now, when I batch with size 4, each batch consists of 4 tensors (samples) of length 7. So it's working correctly.
data = np.random.rand(16,7)
label = np.random.randint(0, 2, size=(16,7))
ds = Dataset.from_dict({"data": data, "label": label}).with_format("torch")
dataloader = DataLoader(ds, batch_size=4)
for batch in dataloader:
    print(batch)

{'data': tensor([[0.1669, 0.0426, 0.8950, 0.5661, 0.5331, 0.0896, 0.0655],
        [0.4167, 0.6302, 0.8909, 0.7454, 0.2768, 0.5057, 0.4715],
        [0.7749, 0.5724, 0.7707, 0.8365, 0.5864, 0.0931, 0.3090],
        [0.9752, 0.2727, 0.1433, 0.6041, 0.2172, 0.5122, 0.3490]]), 'label': tensor([[1, 1, 0, 1, 0, 0, 1],
        [1, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0, 1],
        [1, 1, 0, 0, 0, 1, 0]])}
{'data': tensor([[0.0481, 0.6371, 0.2640, 0.8596, 0.1547, 0.9490, 0.6544],
        [0.2074, 0.4172, 0.1228, 0.5960, 0.4128, 0.8995, 0.5214],
        [0.5998, 0.7333, 0.1849, 0.4986, 0.0315, 0.0149, 0.2415],
        [0.2617, 0.9657, 0.2709, 0.6876, 0.7811, 0.2927, 0.6612]]), 'label': tensor([[0, 0, 1, 1, 1, 0, 0],
        [1, 1, 0, 1, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 1],
        [1, 0, 0, 0, 0, 0, 1]])}
{'data': tensor([[0.3727, 0.3212, 0.9347, 0.9349, 0.3552, 0.0577, 0.5582],
        [0.4870, 0.5731, 0.8875, 0.2646, 0.7427, 0.6187, 0.8907],
        [0.3159, 0.7686, 0.6664, 0.0537

In [ ]:
label = np.random.randint(0, 2, size=(16,7))
label = np.ones((16,7))

In [ ]:
label.shape

(16, 7)

In [ ]:
np.nonzero(label)

(array([ 0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  2,  2,  2,
         2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,  4,  4,  4,
         4,  5,  5,  5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  6,  6,  7,  7,
         7,  7,  7,  7,  7,  8,  8,  8,  8,  8,  8,  8,  9,  9,  9,  9,  9,
         9,  9, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 12,
        12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 14, 14, 14, 14,
        14, 14, 14, 15, 15, 15, 15, 15, 15, 15]),
 array([0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0,
        1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1,
        2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2,
        3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3,
        4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4,
        5, 6]))

In [ ]:
def compute_metrics(eval_pred):
    # Define something here
    return {'my_custom_metric': None}

In [ ]:
from transformers import EarlyStoppingCallback, Trainer, TrainingArguments

training_args = TrainingArguments(
    # Save model checkpoints. # For logging and saving model checkpoints, each 'step' refers to one update of the model's weights.
    # By default, this is once per batch of data, but this isn't necessarily true with gradient accumulation.
    # Typically, we only run validation once per epoch, so there's not much benefit to saving checkpoints more than once per epoch.
    # The only time it's beneficial is if your code is prone to crashing - saving more often means you lose less progress when this happens.
    output_dir='./train_results',
    save_strategy='epoch', # save model checkpoints at the end of each epoch
    eval_strategy='epoch',  # run validation at the end of each epoch

    # Loading the best model state based on the desired metric
    load_best_model_at_end=True,  # load the best model found during training
    # metric_for_best_model="eval_loss",  # Metric to monitor for the best model
    metric_for_best_model='my_custom_metric', # this needs to match a key in the returned dictionary from compute_metrics()
    greater_is_better=True, # higher value for 'my_custom_metric' is better

    # Logs report training and validation metrics, which may be visualized (including in real time) using TensorBoard.
    logging_dir='./train_logs',
    logging_strategy='steps',
    logging_steps=100, # frequency of logging - you need to consider your dataset size and your batch size/gradient accumulation

    per_device_train_batch_size=8, # batch size per device (CPU core/GPU/etc.)
    per_device_eval_batch_size=8, # batch size per device (CPU core/GPU/etc.)

    # Training hyperparameters
    num_train_epochs=10, # max epochs
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,  # This is the validation dataset
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],  # Stop after 3 epochs without improvement
    compute_metrics=compute_metrics,  # custom function to compute validation metrics
)

trainer.train()